In [ ]:
import librosa
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy.signal as signal
from typing import List, Tuple, Callable
from notes import note

In [ ]:
def clip(a: np.array, threshold: float, both: bool = True) -> np.array:
    max = np.max(a)
    a = a / max
    a = np.where(a < threshold, a, threshold)
    if both:
        a = np.where(a > -threshold, a, -threshold)
    clipped = a * max
    return clipped

In [ ]:
def delay(a: np.array, amount: int) -> np.array:
    return a + 0.5* np.concatenate([np.zeros(amount), a])[:-amount]

In [ ]:
def normalize(a: np.array) -> np.array:
    return a / np.max(a)

In [ ]:
def get_signal(frequency: float, function: Callable = np.sin, part: float = 1, samplerate: int = 44100) -> np.array:
    n = samplerate
    t = np.linspace(0, 1, samplerate)
    wave = function(t*2*np.pi*frequency)
    return wave[:int(n*part)]

In [ ]:
def chord(notes: List[str], function: Callable = np.sin, part: float = 1, samplerate: int = 44100) -> np.array:
    n = samplerate
    wave = np.zeros(samplerate)
    for note_name in notes:
        wave = wave + get_signal(note[note_name], function)
    return normalize(wave)[:int(n*part)]

In [ ]:
def melody(notes: List[Tuple[str, float]], function: Callable = np.sin, samplerate: int = 44100) -> np.array:
    """The notes should be a list of tuples. If only a single note (str) is found, it is converted to (str, 1)."""
    convert = lambda n: (n, 1) if type(n) == str else n
    notes = [convert(n) for n in notes]
    melody = np.concatenate([get_signal(note[n], part=p) for n, p in notes])
    return melody

In [ ]:
def play(a: np.array, samplerate: int = 44100, volume: float = 0.2, repeat: int = 1):
    wave = np.tile(normalize(a)*volume, repeat)
    return ipd.Audio(wave, rate=samplerate, autoplay=True, normalize=False)

In [ ]:
c = chord(["C2", "E4", "G4"])
am = chord(["C2", "A3",  "E4"])
f = chord(["F3", "A4", "C4"])
g = chord(["G3", "B4", "D4"])
left = np.concatenate([c, am, f, g])

c  = chord(["C2"], signal.square)*0.3 + 0*chord(["C2"], signal.square) + chord(["C2"])
am = chord(["E2"], signal.square)*0.3 + 0*chord(["E2"], signal.square) + chord(["E2"])
f  = chord(["F2"], signal.square)*0.3 + 0*chord(["F2"], signal.square) + chord(["F2"])
g  = chord(["D2"], signal.square)*0.3 + 0*chord(["D2"], signal.square) + chord(["D2"])
right = np.concatenate([c, am, f, g]) * 0.4

left, right = (left*0.9 + right*0.2), (left*0.1 + right*0.5)
left, right = clip(left, 0.3)*0.5 + left*0.3, 0.5*right + clip(right, 0.1) * 0.5

song = np.c_[left, right]
# song = np.concatenate([song, clip(song, 0.9), clip(song, 0.8), clip(song, 0.7), clip(song, 0.6), clip(song, 0.5), clip(song, 0.4), clip(song, 0.3), clip(song, 0.2), clip(song, 0.1), clip(song, 0.05)])
fig, axs = plt.subplots(2, 1, figsize=(64,16))
plt.ylim((-1, 1))
axs[0].plot(left)
axs[1].plot(right)
play([left, right], repeat=6, volume=0.08)

In [ ]:
vol1 = np.sum(right)
clipped = clip(right, 0.1)
vol2 = np.sum(clipped)
print(vol1, vol2)

In [ ]:
librosa.output.write_wav("out/song.wav", song.T, 44100)

In [ ]:
tooth = signal.sawtooth(np.linspace(0, 2*np.pi, 1000))
clipped = clip(tooth, 0.1, both=True)
plt.plot(clipped)

## Stranger Things

In [ ]:
c2 = chord(["C2"], part=0.3)
e2 = chord(["E2"], part=0.3)
g2 = chord(["G2"], part=0.3)
b2 = chord(["B2"], part=0.3)
c3 = chord(["C3"], part=0.3)
main_melody = np.concatenate([c2, e2, g2, b2, c3, b2, g2, e2])
main_melody = clip(main_melody, 0.9)
main_melody = np.tile(main_melody, 10)
play(main_melody)

In [ ]:
librosa.output.write_wav("out/stranger_things.wav", main_melody, 44100)

## Fairy Tail

### Melody

In [ ]:
m_bar1 = melody([("D5", 1/4), ("E5", 1/8), ("D5", 1/8), ("C5", 1/4), ("A4", 1/4), ("G4", 1/4), ("A4", 1/8), ("C5", 1/8), ("D5", 1/4), ("C5", 1/4)])
m_bar2 = melody([("D5", 1/4), ("E5", 1/8), ("D5", 1/8), ("C5", 1/4), ("A4", 1/4), ("G4", 1/4), ("A4", 1/8), ("C5", 1/8), ("F5", 1/4), ("E5", 1/4)])
m_bar3 = melody([("F5", 1/4), ("E5", 1/8), ("D5", 1/8), ("E5", 1/4), ("D5", 1/8), ("C5", 1/8), ("A4", 1/8), ("C5", 1/8), ("D5", 1/8), ("C5", 1/8), ("F5", 1/4), ("E5", 1/4)])
fairy_melody = np.concatenate([m_bar1, m_bar2, m_bar1, m_bar3])
play(fairy_melody)

### Bass

In [ ]:
b_bar1 = np.concatenate([chord(["D1", "D2"], part=4/4), chord(["D1", "D2"], part=4/4)])
b_bar2 = np.concatenate([chord(["F1", "F2"], part=4/4), chord(["F1", "F2"], part=4/4)])
b_bar3 = np.concatenate([chord(["C2", "C3"], part=4/4), chord(["C2", "C3"], part=4/4)]) * 0.6
b_bar4 = np.concatenate([chord(["Bb1", "Bb2"], part=4/4), chord(["C2", "C3"], part=4/4)]) * 0.6
fairy_bass = np.concatenate([b_bar1, b_bar2, b_bar3, b_bar4]) * 2
play(fairy_bass)

My *ahem* accurate functions sometimes lop of a few elements of the sound array, so I have to equalize the two parts. Since only lopping off occurs, and no additive nonsense, I just have to pad one array a little.

In [ ]:
bass_n = len(fairy_bass)
melody_n = len(fairy_melody)
print(f"{bass_n=}, {melody_n=}")
diff = melody_n - bass_n
print(diff)
if diff < 0: # bass is longer
    fairy_melody = np.pad(fairy_melody, (0, np.abs(diff)))
elif diff > 0: # melody is longer
    fairy_bass = np.pad(fairy_bass, (0, np.abs(diff)))
print(fairy_melody.shape, fairy_bass.shape)

In [ ]:
play(fairy_melody + fairy_bass)

In [ ]:
fairy_tail = np.tile(normalize(fairy_melody + fairy_bass), 8)

In [ ]:
librosa.output.write_wav("out/fairy_tail.wav", fairy_tail, 44100)